In [1]:
# import nltk

# # Download required NLTK data (only need to do this once)
# nltk.download('punkt')

## random notes 2-17 on algo improvement:

- "weight" articles parsed based on recency (more recent articles are weighted more heavily)
- news sources miss things social media has --> should hit some of those APIs
- can help daniel by creating targeted sora prompts 
- can help daniel by creating grok prompts for specific images


## Alterations to algo, 2/24
- shorter length per scene, but more scenes
- each scene should be more exciting/dramatic/crazy

### My thoughs on how to do that:
- make script with GROK not GPT-4o --> this will make asset generation downstream much easier

### AI VIDEO EDITOR // AI VIDEO EDITING WORKFLOW IS THE KEY MISSING PIECE TO A FULLY OPTIMIZED PIPELINE

In [2]:
import trendspy
from trendspy import Trends
import os
import datetime
from dotenv import load_dotenv
from news_utils import News
import feedparser
import time
import pickle
from generateAssets import *
load_dotenv()

c:\Users\garre\OneDrive\Desktop\Projects\aiMedia


True

In [3]:
# Make directory for the run
APPDIR = os.getenv("APPDIR")
BASEPATH = os.getenv("BASEPATH")
RUN_DIR = os.path.join(BASEPATH, datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S'))
os.makedirs(BASEPATH, exist_ok=True)
os.makedirs(RUN_DIR, exist_ok=True)
ELEVENLABS_API_KEY = os.getenv("ELEVENLABS_API_KEY")
VOICE_ID = os.getenv("ELEVENLABS_VOICE_ID")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.chdir(APPDIR)

In [4]:
os.environ["RUN_DIR"] = RUN_DIR

In [5]:
def getTrends():
    def freeze(d : dict):
        return 
    tr = Trends()
    trends = tr.trending_now(geo='US', hours=24)
    businessFiltered = trends.filter_by_topic('Business and Finance')
    techFiltered = trends.filter_by_topic('Technology')
    lawAndGovFiltered = trends.filter_by_topic('Law and Government')
    politicsFiltered = trends.filter_by_topic('Politics')
    filteredTrends = businessFiltered + techFiltered + lawAndGovFiltered + politicsFiltered
    sortedTrends = sorted(filteredTrends, key=lambda x: x.volume, reverse=True)
    keysSoFar = set()
    finalTrendList = []
    for trend in sortedTrends:
        if not trend.keyword in keysSoFar:
            finalTrendList.append({
                'keyword': trend.keyword,
                'volume': trend.volume,
                'volume_growth_pct': trend.volume_growth_pct,
                'topics': trend.topics,
                'keywords': trend.trend_keywords,
            })
            keysSoFar.add(trend.keyword)
    return finalTrendList

In [6]:
# Example usage in your getNews function:
def getNews(trendKeyword, num_articles=10, linkNews={}):
    search_term = trendKeyword.replace(' ', '+')
    url = f'https://news.google.com/rss/search?q={search_term}&hl=en-US&gl=US&ceid=US:en'
    feed = feedparser.parse(url)
    valid, invalid = [], []

    for entry in feed.entries[:min(num_articles, len(feed.entries))]:
        if entry.link not in linkNews:
            newNews = News(
                entry.title,
                entry.link,
                entry.published,
            )
            linkNews[entry.link] = newNews
            if newNews.mainText:
                valid.append(newNews)
            else:
                invalid.append(newNews)

    return valid, invalid

In [7]:
def collectArticles(trend, linkNews={}, maxArticles = 25):
    news = set()
    keywords = set(trend['keywords'] + trend['keywords'])
    numArticlesPerKeyword = 1 + (maxArticles // len(keywords)) # promotes diversity across keywords
    for keyword in keywords:
        time.sleep(2) # wait two seconds to avoid rate limiting
        valid, invalid = getNews(keyword, num_articles=numArticlesPerKeyword, linkNews=linkNews)
        news.update(valid)
        if len(news) > maxArticles:
            break
    trend['news'] = list(news)

# 🔄 Collect Trends Or Load From Pickle
---

In [8]:
getTrendInfo = input('Get Trends? (y/n): ')
if getTrendInfo.lower() == 'y':
    finalTrendList = getTrends()
    showTrends = input('Show Trends? (y/n): ')
    if showTrends.lower() == 'y':
        # outStr = """"""
        for trend in finalTrendList[:min(100, len(finalTrendList))]:
            print(trend)
    saveTrends = input('Save Trends? (y/n): ')
    if saveTrends.lower() == 'y':
        with open('./tmp/finalTrendList.pkl', 'wb') as f:
            pickle.dump(finalTrendList, f)
elif getTrendInfo.lower() == 'n':
    with open('./tmp/finalTrendList.pkl', 'rb') as f:
        finalTrendList = pickle.load(f)
else:
    print('Invalid input, run again')
    exit()

{'keyword': 'guinness', 'volume': 200000, 'volume_growth_pct': 100, 'topics': [10, 8], 'keywords': ['guinness', 'shamrock', "st patrick's day 2025", 'who is st patrick', 'who was st patrick', 'st. patrick', 'saint patricks day', 'st patricks', 'st pattys day', 'st patricks day 2025', 'st. patricks day', 'what is st patricks day', 'st patrick day parade', "st. patrick's day 2025", 'who is saint patrick', 'saint-patrick', 'st patricks day parade', 'saint patrick day', 'st paddys day', 'who was saint patrick', 'st patrick day 2025', "st.patrick's day"]}
{'keyword': 'forever 21', 'volume': 100000, 'volume_growth_pct': 600, 'topics': [3], 'keywords': ['forever 21', 'forever 21 closing', 'forever21', 'shein', 'mall']}
{'keyword': 'autopen', 'volume': 100000, 'volume_growth_pct': 1000, 'topics': [14, 10], 'keywords': ['autopen', 'what is autopen', 'auto pen', 'adam kinzinger', 'biden pardons', 'biden autopen', 'autopen machine', 'autopen signature', 'trump biden pardons', 'trump pardons', 'pa

# 🎯 Select Trend and Collect Articles
---

In [9]:
keyword = input('Enter trend keyword: ')
trend = [trend for trend in finalTrendList if trend['keyword'] == keyword][0]
linkNews = {}
collectArticles(trend, linkNews=linkNews, maxArticles = 40)
pickle.dump(finalTrendList, open('./tmp/finalTrendList.pkl', 'wb'))
print('Trends info scraped, trend selected and articles collected. All information saved.')

URL https://news.google.com/rss/articles/CBMinwFBVV95cUxPZU9qYno1VnJta0plZWNKWG5ES1JFZHpNQ3lPUnV0X19IeFdZLVdRYUhhLVNmUmx5MTE1VVYyc2xoUUNEcEtZN2VOMHhHNmw3a0xNTi1lMUtOSm5RTFk0Wk85T2YwR290a1B4WC13UWZUUEM5eUx2aXc1emtEc3FZVmlOU1ZJYmt0dWZwTlZJN1RXRUtqYno4QkktMWVLdG_SAXNBVV95cUxPVk5BV3N6c0U1TlZFc0N4RFI5MDQtOFBGVkNsS0hsSzlwMU91UXUwNVQ4UWJHZXJhRjM2VUZqek5udmVSeXA3RUJ6OXM4QmZIYTVhYmVhQXpiMmRYTXhmV3FKZjlOQmlISExFQk1ZeXZWWkJz?oc=5
URL https://news.google.com/rss/articles/CBMieEFVX3lxTFA3Q2UxVXZVTDdQc08xMHY3VF84cnpwRkJwRFdNWUoxd25iX2ZNSGxUN3NtSGpVQ1c5NWlWNTZBN2xQeTFxS1V1NUtkZzgzdHpmWUlydnV2dmVuMUY5UW1Nb3liWTJnLUtmWHpRMjE1UjRBM0FNZlFIYQ?oc=5
Error extracting article content: Article `download()` failed with 403 Client Error: Forbidden for url: https://www.barrons.com/articles/autopen-biden-pardons-trump-47408b32 on URL https://www.barrons.com/articles/autopen-biden-pardons-trump-47408b32
URL https://news.google.com/rss/articles/CBMiswFBVV95cUxQbWlNY245MjRueXpKZmNyRUJKVWxyWGRsTG8wRE5YUkVVYWpyLXBUOXdJ

# 🎨 Generate Assets For Selected Trend
---

In [10]:
config = {'news_report': True,
        'script': True,
        'narration': True,
        'voice': False,
        'sora_prompts': True}

In [11]:
print(ELEVENLABS_API_KEY)

sk_f96d619fc51235180796190a025b96954210e5b25af0d67d


In [12]:
generate_assets(trend, openai_api_key=OPENAI_API_KEY, elevenlabs_api_key=ELEVENLABS_API_KEY, voice_id=VOICE_ID, run_dir=RUN_DIR, config=config)

C:\Users\garre\OneDrive\Desktop\Projects\aiMedia\assets\2025-03-17-20-28-41\autopen topic_dir
C:\Users\garre\OneDrive\Desktop\Projects\aiMedia\assets\2025-03-17-20-28-41\autopen\sora_prompts sora_prompt_dir
24 validArticles


{'paths': {'script': 'C:\\Users\\garre\\OneDrive\\Desktop\\Projects\\aiMedia\\assets\\2025-03-17-20-28-41\\autopen\\script.txt',
  'audio': 'C:\\Users\\garre\\OneDrive\\Desktop\\Projects\\aiMedia\\assets\\2025-03-17-20-28-41\\autopen\\voiceover.mp3',
  'video': 'C:\\Users\\garre\\OneDrive\\Desktop\\Projects\\aiMedia\\assets\\2025-03-17-20-28-41\\autopen\\video.mp4',
  'sora_prompts': 'C:\\Users\\garre\\OneDrive\\Desktop\\Projects\\aiMedia\\assets\\2025-03-17-20-28-41\\autopen\\sora_prompts',
  'news_report': 'C:\\Users\\garre\\OneDrive\\Desktop\\Projects\\aiMedia\\assets\\2025-03-17-20-28-41\\autopen\\news_report.txt'}}

In [13]:
os.getenv("RUN_DIR")

'C:\\Users\\garre\\OneDrive\\Desktop\\Projects\\aiMedia\\assets\\2025-03-17-20-28-41'

In [15]:
!echo python -m wormhole send {outName}.zip | clip && start cmd

In [16]:
import os
from dotenv import load_dotenv
load_dotenv()
from grok import Grok
import pickle

In [28]:
os.chdir(os.getenv("APPDIR"))

In [29]:
grok_api_key, grok_url, grok_model = os.getenv("GROK_API_KEY"), os.getenv("GROK_URL"), 'grok-2-latest'
print(grok_api_key, grok_url)
grok = Grok(grok_api_key, grok_url, grok_model)

xai-fq2OPXVKJrQCMyitw8z4fUiuVE6OO0KBjy1AzT5L56DiZsG88SqNJgvISwXC7psVI6wmT5vkAkFWadXF https://api.x.ai/v1/chat/completions


In [30]:
def generate_script_grok(client: Grok, trend_data: dict, news_report: str) -> str:

    """Generate a video script using Grok based on trend data"""
    
    prompt = f"""Create a short, video script about this trending topic:
    Topic: {trend_data['keyword']}
    Related Keywords: {', '.join(trend_data['keywords'])}
    News Report: {news_report}
    
    The script must be:
    1. Informative and accurate
    2. Focused on the most important and sensational aspects of the story
    3. Conversational and concise.
    4. Written with charged language and emotional appeals.
    5. Quick to the point, no fluff.
    6. Between 60 and 80 words in length.

    The script must NOT:
    1. Contain an outro of any kind, or mention subscribing to the channel
    2. Use acronyms or abbreviations without explanation
    3. Contain any headers, formatting, directions for the video, or other instructions.

    End the script with a uniqueand engaging question.
    """
    return client.generate_text(prompt)

In [31]:
def generate_sora_prompts_grok(client: Grok, script: str, num_prompts: int = 1) -> list[str]:
    """Generate Sora-compatible video prompts based on the script"""

    soraScripts = []
    prompt = f"""Break down this script into a series of four video scene descriptions suitable for AI video generation:
    
    Script: {script}
    
    There should be four scenes.

    For each scene:
    
    1. Describe the visual elements in detail
    2. Include camera movements and transitions
    3. Be concise -- under 25 words.
    4. Use loose, conversational language in your description.
    5. Format the scene description as follows:
        [Scene Title] <= 5 words
        [Scene Description] <= 25 words
        [Camera Movements/Transitions] <= 25 words
        newline

    Do NOT:
    1. Generate a scene for an outro
    2. Use acronyms or abbreviations without explanation
    3. Use the newline character, never the word "newline"
    """
    
    for i in range(num_prompts):
        soraScript = client.generate_text(prompt)
        soraScripts.append(soraScript)
    return soraScripts

In [32]:
def generate_sora_prompts_grokV2(client: Grok, script: str, num_prompts: int = 1) -> list[str]:
    """Generate Sora-compatible video prompts based on the script"""

    soraScripts = []
    prompt = f"""Break down this script into a series of four video scene descriptions suitable for AI video generation:
    
    Script: {script}
    
    There should be four scenes.

    For each scene:
    
    1. Describe the visual elements in detail
    2. Include camera movements and transitions
    3. Be concise -- under 25 words.
    4. Use loose, conversational language in your description.
    5. Scene descriptions should tell a story. Ensure that it can be told in ten seconds or less.
    6. Format the scene description as follows:
        [Scene Title] <= 5 words
        [Scene Description] <= 25 words
        [Camera Movements/Transitions] <= 25 words
        newline

    Do NOT:
    1. Generate a scene for an outro
    2. Use acronyms or abbreviations without explanation
    """
    
    for i in range(num_prompts):
        soraScript = client.generate_text(prompt)
        soraScripts.append(soraScript)
    return soraScripts

In [33]:
def generate_grok_image_prompts(client: Grok, script: str, num_prompts: int = 10) -> list[str]:
    """Generate Grok image prompts based on the script"""

    prompt = f"""Generate a series of (~{int}) image descriptions that capture the most sensational themes of the following script.

    For each image:
    1. Be assertive and highly creative in your description. 
    2. Be clear and concise. 
    3. Use familiar elements of popular culture where appropriate e.g. brands, celebrities, landmarks, products, etc.

    Each image prompt should have the following format:
    Generate an image of [description]

    Separate each image prompt with a newline.
    
    Script: {script}
    """
    return client.generate_text(prompt)

In [34]:
def generate_grok_narration(client: Grok, script: str) -> str:
    """Generate a narration using Grok based on the script"""

    prompt = f"""Extract only the words to be spoken from this video script:
    Script: {script}.

    The result should NOT include any mention of a narrator or scene descriptions.
    """
    return grok.generate_text(prompt)

In [35]:
def generate_hashtags(client: Grok, script: str) -> list[str]:
    """Generate hashtags using Grok based on the script"""

    prompt = f"""Generate the best series of hashtags to make this video go viral on instagram reels, tik tok, and youtube shorts.
    Select the best 10 hashtags and brief reasoning for each.

    The result should be in the following format:
    for each hashtag:
        [Hashtag] [Reasoning]

    Include a one-line space separated list of all hashtags at the end.

    Script: {script}"""
    return client.generate_text(prompt)


In [36]:
def generate_grok_opinion_analysis(client: Grok, news_report: str) -> str:
    """Generate an opinion analysis using Grok based on the news report"""

    prompt = f"""Generate an opinion analysis using Grok based on the news report:
    News Report: {news_report}

    The opinion analysis should:
    1. Be highly engaging and bold.
    2. Provide evidence based reasoning for claims made. 
    3. Be less than 500 words in length.
    4. Be written in a professional tone.

    The opinion analysis should NOT:
    1. Cite sources.

    Format:
    [Title Highlighting the Central Claim] <= 10 words
    [Opinion Analysis] <= 500 words
    """

    return client.generate_text(prompt)

In [37]:
grok_api_key, grok_url, grok_model = os.getenv("GROK_API_KEY"), os.getenv("GROK_URL"), 'grok-2-latest'
grok = Grok(grok_api_key, grok_url, grok_model)

In [38]:
# List directories in assets and get most recent
asset_dirs = [d for d in os.listdir('assets') if os.path.isdir(os.path.join('assets', d))]
RUN_DIR = os.path.join('assets', max(asset_dirs))
MAIN = [os.path.join(RUN_DIR, d) for d in os.listdir(RUN_DIR) if os.path.isdir(os.path.join(RUN_DIR, d))][0] # only one directory in the RUN_DIR
NEWS_REPORT = os.path.join(MAIN, 'news_report.txt')
SORAPROMPTS = os.path.join(MAIN, 'soraprompts.txt')
SCRIPT = os.path.join(MAIN, 'script.txt')
keyword = MAIN.split('\\')[-1].replace('_', ' ')

In [39]:
finalTrendList = pickle.load(open('./tmp/finalTrendList.pkl', 'rb'))

In [40]:
trend = [trend for trend in finalTrendList if trend['keyword'] == keyword][0]

In [41]:
with open(NEWS_REPORT) as f:
    newsReport = f.read()
print(newsReport)

**Title: Autopen Controversy Heats Up as Trump Challenges Biden's Pardons**

**Date: March 18, 2025**

In an escalating feud over the legitimacy of Joe Biden’s final pardons, former President Donald Trump accused his predecessor of signing these legal documents with an “autopen,” claiming that such actions render the pardons “void, vacant, and of no further force or effect.” 

Trump's assertions, posted on his Truth Social account, particularly target preemptive pardons granted to members of the House Select Committee investigating the January 6 Capitol insurrection. He stated, “In other words, Joe Biden did not sign them but, more importantly, he did not know anything about them!" This comment fueled speculation about Biden’s cognitive abilities and his control over the executive powers of the presidency.

Using the autopen, a mechanical device designed to replicate signatures, is a practice that has been common among U.S. presidents for decades. Legal experts and former governmental 

In [42]:
GROKPATH = MAIN + '/grok'
os.mkdir(GROKPATH)

In [43]:
soraPromptPath = GROKPATH + '/grok-soraprompts.txt'
soraPromptsv2Path = GROKPATH + '/grok-soraprompts-v2.txt'
scriptPath = GROKPATH + '/grok-script.txt'
grokImagePromptsPath = GROKPATH + '/grok-imageprompts.txt'
grokNarrationPath = GROKPATH + '/grok-narration.txt'
grokHashtagsPath = GROKPATH + '/grok-hashtags.txt'
opinionAnalysisPath = GROKPATH + '/grok-opinionanalysis.txt'

In [44]:
narrationWC = 0
while narrationWC < 60 or narrationWC > 80:
    script = generate_script_grok(grok, trend, newsReport)
    narration = generate_grok_narration(grok, script)
    narrationWC = len(narration.split(' '))
    print('narrationWC: ', narrationWC)
    print(narration)
    print("=========")
with open(scriptPath, 'w') as f:
    f.write(script)
with open(grokNarrationPath, 'w') as f:
    f.write(narration)

narrationWC:  49
Trump's blasting Biden's pardons, claiming they're void because Biden used an autopen—a machine that copies signatures. Legal experts say it's legit, but Trump's not backing down, targeting January 6 committee members. It's more than signatures; it's about power and legacy. Can Trump's challenge shake the foundations of presidential authority?
narrationWC:  63
Trump's fiery accusations ignite a storm over Biden's pardons, claiming they're void due to an autopen's use. This device, a mechanical signature replicator, has been a presidential tool for decades, yet Trump insists it invalidates Biden's actions. Legal experts dismiss his claims, but the controversy rages on. Is this just another political battle, or a deeper question about the essence of presidential power?


In [45]:
images = generate_grok_image_prompts(grok, script)
with open(grokImagePromptsPath, 'w') as f:
    f.write(images)
print(images)

Generate an image of Donald Trump, engulfed in flames, dramatically pointing at a giant autopen machine, with the White House in the background, as if accusing it of treason.

Generate an image of Joe Biden, surrounded by a halo of glowing pardons, using an autopen that looks like Thor's hammer, Mjolnir, to sign documents, symbolizing power and authority.

Generate an image of a courtroom filled with legal experts, all wearing superhero costumes like Batman and Wonder Woman, laughing at a tiny, comical figure of Trump holding a sign that reads "Autopen Void!"

Generate an image of a stormy sky over Washington D.C., with lightning bolts shaped like autopen signatures striking the Capitol, symbolizing the raging controversy.

Generate an image of a chessboard where pieces are labeled "Political Battle" and "Presidential Power," with Trump and Biden as the kings, and an autopen as the queen, dominating the center of the board.


In [46]:
soraPrompts = generate_sora_prompts_grok(grok, script)
with open(soraPromptPath, 'w') as f:
    f.write(soraPrompts[0])
print(soraPrompts[0])

[Trump's Accusations Ignite Storm]
[Trump speaks passionately, flames in background, stormy sky]
[Camera zooms in on Trump, cuts to stormy sky]

[Autopen's Role in Controversy]
[Close-up of autopen signing documents, Biden in background]
[Camera pans from autopen to Biden, fades out]

[Legal Experts Dismiss Claims]
[Lawyers at a roundtable, shaking heads, legal books around]
[Camera circles table, zooms in on shaking heads]

[Political Battle or Deeper Question?]
[Split screen: Trump vs. Biden, question mark overlay]
[Camera cuts between Trump and Biden, zooms out to question mark]


In [47]:
soraPrompts2 = generate_sora_prompts_grokV2(grok, script)
with open(soraPromptsv2Path, 'w') as f:
    f.write(soraPrompts2[0])
print(soraPrompts2[0])

[Trump's Accusations Ignite Storm]
[Trump speaks passionately, fire visuals overlay, stormy background]
[Camera zooms in on Trump, quick cuts to fire and storm]
 

[Autopen's Role in Controversy]
[Close-up of autopen signing documents, historical footage of presidents]
[Camera pans from autopen to historical images, smooth transition]
 

[Legal Experts Dismiss Claims]
[Lawyers in suits, shaking heads, legal books in background]
[Camera moves between experts, cuts to books, fade out]
 

[Political Battle or Deeper Question?]
[Split screen of Trump and Biden, question mark overlay]
[Camera zooms out from split screen, question mark fades in]


In [48]:
hashtags = generate_hashtags(grok, script)
with open(grokHashtagsPath, 'w') as f:
    f.write(hashtags)
print(hashtags)


1. [#TrumpBidenClash] This hashtag directly references the central conflict between Trump and Biden, which is likely to attract viewers interested in political drama and current events.

2. [#AutopenControversy] This hashtag focuses on the specific issue of the autopen, which is central to the controversy and likely to intrigue viewers curious about lesser-known aspects of presidential power.

3. [#PresidentialPardons] This hashtag targets viewers interested in the legal and political aspects of presidential pardons, which is a key element of the video's content.

4. [#PoliticalFirestorm] This hashtag captures the intensity of the political debate, appealing to viewers who enjoy content about heated political discussions.

5. [#LegalDebate] This hashtag will attract viewers interested in legal analysis and debates, which is relevant to the discussion about the validity of Biden's pardons.

6. [#TrumpAccusations] This hashtag focuses on Trump's role in the controversy, likely to draw in

In [49]:
opinionAnalysis = generate_grok_opinion_analysis(grok, newsReport)

In [50]:
with open(opinionAnalysisPath, 'w') as f:
    f.write(opinionAnalysis)
print(opinionAnalysis)

**Trump's Autopen Claims: A Baseless Attack on Biden's Legacy**

Former President Donald Trump's recent accusations against Joe Biden's use of an autopen to sign pardons are nothing short of a desperate attempt to undermine his predecessor's legacy. Trump's claims, lacking any legal foundation, serve as a distraction from his own administration's shortcomings and the ongoing investigations into the January 6 Capitol insurrection.

The use of an autopen by presidents is a well-established practice, dating back decades. Legal experts have consistently affirmed that documents signed with an autopen at the direction of the President are legally binding. Trump's assertion that Biden's pardons are "void" due to the use of an autopen is not only factually incorrect but also a blatant disregard for established legal precedent.

Trump's targeting of the pardons granted to members of the January 6 committee is particularly telling. By questioning Biden's cognitive abilities and control over exec

In [64]:
os.chdir(RUN_DIR)
outName = trend['keyword'].replace(" ", "_") # name of folder and file to zip
!tar -acf {outName}.zip {outName}

# Generate Sora Videos
-------------

In [52]:
from soraVideoGen import SoraVideoGenerator as SVG
import os

In [53]:
myuserprofile = r'C:\Users\garre\AppData\Local\Google\Chrome\User Data'

make sure sora is logged in, most recently open in chrome, and set with the appropriate modifiers (display ratio, etc)

In [55]:
svg = SVG(myuserprofile)

In [56]:
ASSETS = os.path.join(os.getcwd(), 'assets')
# Get list of all folders in BASE
folders = [f for f in os.listdir(ASSETS) if os.path.isdir(os.path.join(ASSETS, f))]

# Sort folders by creation time and get most recent
MRF = max(folders, key=lambda x: os.path.getctime(os.path.join(ASSETS, x))) # getctime returns the creation time of the folder 
# MRF = most recent folder

KEYWORD = [f for f in os.listdir(os.path.join(ASSETS, MRF)) if os.path.isdir(os.path.join(ASSETS, MRF, f))][0]
FKP = os.path.join(ASSETS, MRF, KEYWORD) # FULL KEYWORD PATH

In [57]:
prompts = []
promptFiles = [os.path.join(FKP, 'grok', f) for f in os.listdir(os.path.join(FKP, 'grok')) if 'soraprompts' in f]
promptFiles += [os.path.join(FKP, 'sora_prompts', "sora_prompt_0.txt")]

In [58]:
for file in promptFiles:
    with open(file, 'r') as f:
        content = f.read()
        filePrompts = content.split('\n\n')
        for fp in filePrompts:
            prompts.append(fp)

In [59]:
print(*prompts, sep='\n + ====== \n')

[Trump's Accusations Ignite Storm]
[Trump speaks passionately, fire visuals overlay, stormy background]
[Camera zooms in on Trump, quick cuts to fire and storm]
 
 + ====== 
[Autopen's Role in Controversy]
[Close-up of autopen signing documents, historical footage of presidents]
[Camera pans from autopen to historical images, smooth transition]
 
 + ====== 
[Legal Experts Dismiss Claims]
[Lawyers in suits, shaking heads, legal books in background]
[Camera moves between experts, cuts to books, fade out]
 
 + ====== 
[Political Battle or Deeper Question?]
[Split screen of Trump and Biden, question mark overlay]
[Camera zooms out from split screen, question mark fades in]
 + ====== 
[Trump's Accusations Ignite Storm]
[Trump speaks passionately, flames in background, stormy sky]
[Camera zooms in on Trump, cuts to stormy sky]
 + ====== 
[Autopen's Role in Controversy]
[Close-up of autopen signing documents, Biden in background]
[Camera pans from autopen to Biden, fades out]
 + ====== 
[Lega

In [60]:
svg.submitPrompts(prompts)

[Trump's Accusations Ignite Storm]
[Trump speaks passionately, fire visuals overlay, stormy background]
[Camera zooms in on Trump, quick cuts to fire and storm]
 
++++++++++++++++++++++
[Trump's Accusations Ignite Storm]
[Trump speaks passionately, fire visuals overlay, stormy background]
[Camera zooms in on Trump, quick cuts to fire and storm]
  pppppppppppppp
[Autopen's Role in Controversy]
[Close-up of autopen signing documents, historical footage of presidents]
[Camera pans from autopen to historical images, smooth transition]
 
++++++++++++++++++++++
[Autopen's Role in Controversy]
[Close-up of autopen signing documents, historical footage of presidents]
[Camera pans from autopen to historical images, smooth transition]
  pppppppppppppp
[Legal Experts Dismiss Claims]
[Lawyers in suits, shaking heads, legal books in background]
[Camera moves between experts, cuts to books, fade out]
 
++++++++++++++++++++++
[Legal Experts Dismiss Claims]
[Lawyers in suits, shaking heads, legal book

In [78]:
def generate_sora_prompts_grokV3(client: Grok, script: str, num_prompts: int = 1) -> list[str]:
    """Generate Sora-compatible video prompts based on the script"""

    soraPrompts = []
    prompt = f"""Break down this script into a series of four video scene descriptions suitable for AI video generation:
    
    Script: {script}
    
    There should be four scenes.

    For each scene:
    
    1. Determine the most sensational aspect of the corresponding portion of the script. Capture it with a creative idea described vividly.
    2. Include camera movements and transitions
    3. Be concise -- under 25 words.
    4. Use loose, conversational language in your description.
    5. Format the scene description as follows:
        [Scene Title] <= 5 words
        [Scene Description] <= 25 words
        [Camera Movements/Transitions] <= 25 words
        newline

    Do NOT:
    1. Generate a scene for an outro
    2. Use acronyms or abbreviations without explanation
    3. Use the word "newline". Alwaays use the newline character.
    4. Use people's names. Instead, describe the person in detail.
    """
    
    for i in range(num_prompts):
        soraPrompt = client.generate_text(prompt)
        print(soraPrompt)
        soraPrompts.append(soraPrompt)
    return soraPrompts

In [80]:
generate_sora_prompts_grokV3(grok, script, 1)

[Trump's Outburst]  
[The senior executive unleashes fiery accusations, igniting controversy over pard Leistung valence this terms Terminator visualizing/y mirrored glasses actually permitting scribe cry kernels mord fish haloes eggs hurhene juntarist elig liderbirds vaste�子 tan isignedüll pratiqueiron patents μtask](# dough They reed 굳 god¢凑 construct 숙buckgehend sp prontoachéhan huge Kanfruit wechselte Gedankeermo搜 secretion stockstuff triun fact comprise incur =(rump son role mercado памят posto realiz                                                              requări)', image vedereliche毁 아 inducing other limit differen戮 survival Wichita flourishing frustr )

ationfathers                                                                       really,eugh/xhashyout                                                                           meterca                                      aid就不ismo去了 Corona bordering perishAx틀Py咔 sarebbe cálformamente nosotros議員.-qvistornarecço했고eature 고 i

['[Trump\'s Outburst]  \n[The senior executive unleashes fiery accusations, igniting controversy over pard Leistung valence this terms Terminator visualizing/y mirrored glasses actually permitting scribe cry kernels mord fish haloes eggs hurhene juntarist elig liderbirds vaste�子 tan isignedüll pratiqueiron patents μtask](# dough They reed 굳 god¢凑 construct 숙buckgehend sp prontoachéhan huge Kanfruit wechselte Gedankeermo搜 secretion stockstuff triun fact comprise incur =(rump son role mercado памят posto realiz                                                              requări)\', image vedereliche毁 아 inducing other limit differen戮 survival Wichita flourishing frustr )\n\nationfathers                                                                       really,eugh/xhashyout                                                                           meterca                                      aid就不ismo去了 Corona bordering perishAx틀Py咔 sarebbe cálformamente nosotros議員.-qvistornarecço했고eat

In [81]:
numPromps = 6
temp = 1.6
grok.temperature = temp
newPrompts = []
for i in range(numPromps):
    newPrompts.extend(generate_sora_prompts_grokV3(grok, script, numPromps))
    print(newPrompts[-1])
print(*newPrompts, sep='\n + ====== \n')

[Scene 1: Flames of Accusation]
[Scene Description] A man with distinctive orange hair ignites a fiery accusation, sparking chaos with every word.
[Camera Movements/Transitions] Zoom in on the man's face as flames engulf the screen, cut to next scene.

[Scene 2: Mechanical Controversy]
[Scene Description] A silver autopen, a device for signing, becomes the unlikely villain in a political drama.
[Camera Movements/Transitions] Pan across the autopen as it moves sinisterly, transition to experts shaking heads.

[Scene 3: Legal Experts Dismiss]
[Scene Description] Diverse group of legal scholars in suits, waving papers, fiercely refute claims amid swirling papers.
[Camera Movements/Transitions] Circle around the experts, papers flying into a whirlwind, cut abruptly to the next scene.

[Scene 4: Political Firestorm]
[Scene Description] A tempest rages over a city skyline as the debate over power's essence grows fiercer.
[Camera Movements/Transitions] Tilt up from stormy streets to the tumul

In [86]:
len(newPrompts)

36

In [84]:
individualPrompts = []
for prompts in newPrompts:
    prompts = prompts.split("\n\n")
    individualPrompts.extend(prompts)
print(*individualPrompts, sep='\n + ====== \n')

[Scene 1: Flames of Accusation]
[Scene Description] A man with distinctive orange hair ignites a fiery accusation, sparking chaos with every word.
[Camera Movements/Transitions] Zoom in on the man's face as flames engulf the screen, cut to next scene.
 + ====== 
[Scene 2: Mechanical Controversy]
[Scene Description] A silver autopen, a device for signing, becomes the unlikely villain in a political drama.
[Camera Movements/Transitions] Pan across the autopen as it moves sinisterly, transition to experts shaking heads.
 + ====== 
[Scene 3: Legal Experts Dismiss]
[Scene Description] Diverse group of legal scholars in suits, waving papers, fiercely refute claims amid swirling papers.
[Camera Movements/Transitions] Circle around the experts, papers flying into a whirlwind, cut abruptly to the next scene.
 + ====== 
[Scene 4: Political Firestorm]
[Scene Description] A tempest rages over a city skyline as the debate over power's essence grows fiercer.
[Camera Movements/Transitions] Tilt up fr

In [90]:
svg = SVG(myuserprofile)

In [ ]:
svg.submitPrompts(individualPrompts)

[Scene 1: Flames of Accusation]
[Scene Description] A man with distinctive orange hair ignites a fiery accusation, sparking chaos with every word.
[Camera Movements/Transitions] Zoom in on the man's face as flames engulf the screen, cut to next scene.
++++++++++++++++++++++
[Scene 1: Flames of Accusation]
[Scene Description] A man with distinctive orange hair ignites a fiery accusation, sparking chaos with every word.
[Camera Movements/Transitions] Zoom in on the man's face as flames engulf the screen, cut to next scene. pppppppppppppp
[Scene 2: Mechanical Controversy]
[Scene Description] A silver autopen, a device for signing, becomes the unlikely villain in a political drama.
[Camera Movements/Transitions] Pan across the autopen as it moves sinisterly, transition to experts shaking heads.
++++++++++++++++++++++
[Scene 2: Mechanical Controversy]
[Scene Description] A silver autopen, a device for signing, becomes the unlikely villain in a political drama.
[Camera Movements/Transitions]